Сравнение подозрительного файла с ранее проанализированными образцами или образцами, хранящимися в публичном либо частном хранилище, может дать представление о семействе вредоносных программ, их характеристиках и сходстве с предварительно проанализированными образцами.

Хотя криптографические хеш-функции (MD5/SHA1/SHA256) являются отличным методом для обнаружения идентичных образцов, они не помогают в идентификации схожих образцов. Очень часто авторы вредоносных программ меняют мелкие аспекты вредоносных программ, что полностью меняет значение хеш-функции. 

Нечеткое хеширование – отличный способ сравнить файлы на схожесть. 

[Ssdeep](https://ssdeep-project.github.io/ssdeep/) – полезный инструмент для создания нечеткого хеша для образца, и он также помогает в определении процентного сходства между
образцами. Этот метод полезен при сравнении подозрительного файла с образцами из хранилища для идентификации похожих. Это может помочь определить образцы, принадлежащие к одному семейству вредоносных программ или к одной и той же группе субъектов.

In [2]:
#apt-get -y install libfuzzy-dev
#apt-get install ssdeep
#pip install ssdeep

Чтобы определить нечеткий хеш образца, выполните следующую команду:

In [1]:
!ssdeep test

ssdeep,1.1--blocksize:hash:hash,filename
96:RZTmB6WBilMGIlZIpMRiUHsTgZ6uQ7fTuB3eoBqSitrqN:RZioWQMZMpLfgZ8+4osSi,"/home/dmf/new_github/infosec/re-tools/test"


Чтобы продемонстрировать использование нечеткого хеширования, рассмотрим в качестве примера директорию, состоящую из трех образцов вредоносного ПО. 

В следующем фрагменте кода видно, что все три файла имеют совершенно разные значения хеш-функций MD5:

In [2]:
!ls

main_01.c  main_03.c  test_01  test_03
main_02.c  test       test_02  нечеткое хеширование.ipynb


In [5]:
!md5sum test_01 test_02 test_03

7c2c629a6b03a1db0ad5f168c29317a3  test_01
f8da20d939b7f10d2de94c22cf28f7dd  test_02
aca771dc841b399b8dfb66fcdaae130d  test_03


Режим изящного сравнения (опция -p) в ssdeep может использоваться для определения процентного сходства. Из трех образцов два имеют сходство 93%, что предполагает, что они, вероятно, принадлежат к одному и тому же семейству вредоносных программ:

In [6]:
!ssdeep -pb test_01 test_02 test_03

test_02 matches test_03 (93)

test_03 matches test_02 (93)



Как показано в предыдущем примере, криптографические хеш-функции не помогли установить связь между образцами, тогда как метод нечеткого хеширования выявил сходство. 

Можно запустить ssdeep для каталогов и подкаталогов, содержащих вредоносные образцы, используя рекурсивный режим (-r):

In [7]:
#ssdeep -lrpa samples/

В следующем примере ssdeep-хеши всех файлов были перенаправлены в текстовый файл (all_hashes.txt), а затем подозрительный файл (test_03) сопоставляется со всеми хешами в файле: 

In [8]:
!ssdeep test_01 test_02 test_03 > all_hashes.txt

In [9]:
!cat all_hashes.txt

ssdeep,1.1--blocksize:hash:hash,filename
96:RZTWB6WBilMGIlZIpMRiUHs9ga7fNuB3eoBqSitrqN:RZ6oWQMZMpLrI4osSi,"/home/dmf/new_github/infosec/re-tools/test_01"
96:RIXTekB6WBn+ZZS9+MRSM8vKk7fAuBWeoBqSAXLgj:RIX9oWgZS9+bz7xosSA,"/home/dmf/new_github/infosec/re-tools/test_02"
96:RIXTtB6WBn+ZZSg+MRSM8vKk7fXuBWeoBqSAXLgj:RIXRoWgZSg+bzyxosSA,"/home/dmf/new_github/infosec/re-tools/test_03"


В следующем фрагменте кода видно, что подозрительный файл (test_03) идентичен test_03 (соответствие – 100%) и имеет сходство 93% с test_02. Можно использовать этот метод для сравнения любого нового файла с хешами ранее проанализированных образцов:

In [10]:
!ssdeep -m all_hashes.txt test_03

/home/dmf/new_github/infosec/re-tools/test_03 matches all_hashes.txt:/home/dmf/new_github/infosec/re-tools/test_02 (93)
/home/dmf/new_github/infosec/re-tools/test_03 matches all_hashes.txt:/home/dmf/new_github/infosec/re-tools/test_03 (100)


В Python нечеткий хеш может быть вычислен с использованием python-ssdeep:

In [12]:
#!pip3 install ssdeep

  Using cached ssdeep-3.4.tar.gz (110 kB)
  Created wheel for ssdeep: filename=ssdeep-3.4-cp37-cp37m-linux_x86_64.whl size=40172 sha256=0fbc5a279e91bca0366f3457f125ebd022f68795ce5cac4a8bfb2c5568e975d7
  Stored in directory: /home/dmf/.cache/pip/wheels/4a/a4/39/73512b7516c4a7245d806e51fd10d071d26f50095604f4f822
Successfully built ssdeep


In [15]:
import ssdeep
hash1 = ssdeep.hash_from_file('test_03')
print(hash1)

96:RIXTtB6WBn+ZZSg+MRSM8vKk7fXuBWeoBqSAXLgj:RIXRoWgZSg+bzyxosSA


In [16]:
hash2 = ssdeep.hash_from_file('test_02')
print(hash2)

96:RIXTekB6WBn+ZZS9+MRSM8vKk7fAuBWeoBqSAXLgj:RIX9oWgZS9+bz7xosSA


In [17]:
ssdeep.compare(hash1, hash2)

93